In [ ]:
import healpy as hp

In [ ]:
cd output/0000

In [ ]:
m = hp.read_map("simonsobs_noise_uKCMB_ST0_UHF1_nside1024_0000_1_of_1.fits", (0,1,2))

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(m[0], min=-100,max=100)

In [ ]:
from so_models_v3 import SO_Noise_Calculator_Public_v3_1_1 as so_models

import matplotlib
matplotlib.rc('text', usetex=False)
fontProperties = {
                  'weight' : 'normal', 'size' : 16}
import matplotlib.pyplot as plt

####################################################################
####################################################################
##                   demonstration of the code
####################################################################

mode=1 # baseline
fsky=1
ellmax=1e4
el=None

dset_label = 'LAT\\_V3.1'
lat = so_models.SOSatV3point1(mode, el=el )

print(dset_label)
bands = lat.get_bands()
print("band centers: ", lat.get_bands(), "[GHz]")
print("beam sizes: "  , lat.get_beams(), "[arcmin]")
N_bands = len(bands)

ell, N_ell_LA_T_full,N_ell_LA_P_full = lat.get_noise_curves(
    fsky, ellmax, 1, full_covar=False, deconv_beam=False, rolloff_ell=50)

WN_levels = lat.get_white_noise(fsky)**.5

N_ell_LA_T  = N_ell_LA_P_full / 2
N_ell_LA_P  = N_ell_LA_P_full

print("white noise levels: "  , WN_levels, "[uK-arcmin]")


In [ ]:
npix = len(m[0])
nside = hp.npix2nside(npix)
sky_fraction=(m[0]>-1e30).sum()/npix

In [ ]:
sky_fraction

In [ ]:
from mapsims import SONoiseSimulator, SOChannel
sim = SONoiseSimulator(telescopes=["SA"], nside=1024, scanning_strategy="classical", num=134,
                       rolloff_ell=50, hitmap_version="v0.2", full_covariance=True,
                       SA_number_LF=0.4, SA_number_MF=1.6, SA_number_UHF=1,
                       apply_beam_correction=False)

In [ ]:
m = sim.simulate(tube="ST0")

In [ ]:
cl = hp.anafast(m[0], use_pixel_weights=True, lmax=1024)# / sky_fraction

In [ ]:
import mapsims
from astropy import units as u

In [ ]:
bands

In [ ]:
mapsims.SONoiseSimulator

In [ ]:
ylim = (1e-6, 1e-1)
i_tube = 0
for (tube, chs) in [("STO", ["UHF1"])]:
    if tube.startswith("S"):
    
        for i, ch in enumerate(chs):
            freq = mapsims.so_utils.parse_channels("SA_"+ch)[0].center_frequency.value
            plt.loglog(ell, N_ell_LA_P[bands.searchsorted(freq)])
            plt.ylabel(r"N_ell [muK^2]", fontsize=16)
        for pol in [1,2]:
            for i, ch in enumerate(chs):
                len_cl = len(cl)
                pixwin = hp.pixwin(8192 if len_cl>10000 else (len_cl)/3, lmax=len_cl-1)
                plt.loglog(cl[pol])

                plt.ylim(*ylim)
                plt.title(tube + ": " + ch, fontsize=16)
        i_tube += 1